OPENSTREETMAP DATA CASE STUDY

I have used DETROIT/MICHIGAN OSM XML data for the purpose of this project. 
The below code is to convert the OSM XML file to JSON file so that it can be imported into a MongoDB database.
The procedure to import the JSON file into the MongoDB database was carried out in the Windows command prompt through the command:
$ mongoimport --db openstreetmap --collection detroit --file detroit_michigan.osm.json

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]') # To eliminate problematic characters from our data

# Standard keys for 'way' and 'node' tags
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

# The mapping dictionary is used to convert address streets' shorthand notations to full names for uniformity.
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Rd": "Road",
            "Ave": "Avenue",
            "Ct": "Court",
            "Ct.": "Court",
            "Pl": "Place",
            "Pl.": "Place",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Sq.": "Square",
            "Sq": "Square",
            "Tr": "Trail",
            "Tr.": "Trail",
            "Pw" : "Parkway",
            "Pw.": "Parkway",
            "Co" : "Commons",
            "Co.": "Commons",
            }


def shape_element(element):
    # Defing the initial structure of the element, although not required.'node' variable is for both 'node' and 'way' here
    node = {'id':'','visible':'','type':'', 'pos':[],
            'created':{'changeset':'','user':'','version':'','uid':'','timestamp':''} }
    
    if element.tag == "node" or element.tag == "way" :
        
        if element.tag == 'node':
            node['pos'] = [element.attrib['lat'],element.attrib['lon']] # Set 'pos' key as list [latitude, longitude]
            
        if 'visible' in element.attrib:
            node['visible'] = element.attrib['visible']
            
        node['type'] = element.tag
        node['id'] = element.attrib['id']
        
        # Adding key/value pairs from main attributes
        for item in CREATED:
            node['created'][item] = element.attrib[item]
            
        # Way tags also contain node_refs
        if element.tag == 'way':
            node['node_refs'] = []
            
        # Iterating inside the main element
        for item in element.iter():
            
            # For 'tag' elements only
            if item.tag == 'tag':
                k = item.attrib['k']
                split_k = k.split(':') # Splitting from ':' to check if 'addr' or not
                
                # For checking for address street for problemchars
                if not problemchars.search(k) and len(split_k) <= 2:
                    
                    if split_k[0] == 'addr':
                        if 'address' not in node:
                            node['address'] = {}   # Create address subkey if it doesn't exist
                        
                        if k == 'addr:street':
                            st = item.attrib['v'].split()[-1]   # Shorthand notation for street type.
                            
                            # Replace the shorthand notation with the full type.
                            if st in mapping:
                                node['address']['street'] = item.attrib['v'].replace(st,mapping[st])
                                # Separate attribute for street type for simpilfying querying based on type of street
                                node['address']['street_type'] = mapping[st]
                        else:   
                            # Making 'address.state' uniform for 'Michigan' state
                            if split_k[1] == 'state' and (item.attrib['v'] in ['MI','mi','Mchigan','MICHIGAN']):
                                node['address']['state'] = 'Michigan'
                            else:
                                node['address'][split_k[1]] = item.attrib['v']
                    else:
                        node[k] = item.attrib['v']
                        
            # For 'nd' elements only. Occurs in 'way' elements only.
            elif item.tag == 'nd':
                node['node_refs'].append(item.attrib['ref'])
                
        # Cleaning the 'lanes' attribute, if it exists
        if 'lanes' in node:
            node['lanes'] = str(node['lanes']).replace(' ','')
            if ';' in node['lanes']:
                node['lanes'] = node['lanes'].split(';')
        return node
    else:
        return None

def get_element(osm_file, tags=('node', 'way')):
    
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def process_map(file_in, pretty = False):
    
    file_out = "{0}.json".format(file_in)
    
    with codecs.open(file_out, "w") as fo:
        
        for i, elem in enumerate(get_element(file_in)):
            el = shape_element(elem)
            if el:
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
            elem.clear()

# Calling the function to carry out the parsing            
process_map('D:\DANDProjects\detroit_michigan.osm')

--------------------------------------------------------------------------------------------------------------------------------
From here we begin querying and checking the data for correctness and completeness.

Our database name is 'openstreetmap' and collection name is 'detroit'.
The database is assigned to the variable 'db' and the collection to the variable 'collection'.

In [18]:
from pymongo import MongoClient
client = MongoClient()
db = client.openstreetmap
collection = db.detroit_2

Size of the Collection (in bytes)

In [3]:
print db.command('collstats','detroit')['size']

1077358667.0


In [17]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

Total number of documents

In [5]:
print collection.count()

4226232


Total number of nodes:

In [6]:
print collection.find({'type':'node'}).count()

3838133


Total number of ways:

In [4]:
print collection.find({'type':'way'}).count()

387910


Number of tags which are neither nodes, nor ways:

In [42]:
not_way_or_node = collection.find({'type':{'$nin':['node','way']}})

In [46]:
print not_way_or_node.count()

189


Number of nodes which are visible and which are not:

In [10]:
print collection.find({'visible':'false'}).count()

0


In [14]:
print collection.find({'visible':'true'}).count()

0


Since we cannot see any document with 'visible' key set to 'true' or 'false', we remove the field from our documents. This could have been avoided by modifying the code for parsing.

In [24]:
remove_visible = collection.update_many({'visible':''}, { '$unset' : {'visible': 1}})
pp.pprint(collection.find_one())

{   u'_id': ObjectId('5809a488251362cbcb23c09f'),
    u'created': {   u'changeset': u'41510088',
                    u'timestamp': u'2016-08-17T11:15:01Z',
                    u'uid': u'1679',
                    u'user': u'andrewpmk',
                    u'version': u'4'},
    u'highway': u'traffic_signals',
    u'id': u'11753030',
    u'pos': [u'42.2547374', u'-83.0172501'],
    u'type': u'node'}


Unwanted types are present due to second level tag's 'k' attribute:

In [48]:
print collection.distinct('type')

[u'node', u'public', u'broad_leaved', u'conifer', u'audio', u'way', u'multipolygon', u'route', u'trap', u'boundary', u'gas', u'oil', u'preserved', u'civil']


Removing the documents wuth type other than 'node' or 'way':

In [7]:
collection.delete_many({'type':{'$nin':['node','way']}})

New document count after deleting the unrequired documents:

In [8]:
new_length = collection.find().count()
print new_length

4226043


Sample document of the type 'way':

In [34]:
pp.pprint(collection.find_one({'type':'way'}))

{   u'_id': ObjectId('58061988ed3120e54385c2b9'),
    u'created': {   u'changeset': u'25545638',
                    u'timestamp': u'2014-09-19T17:22:21Z',
                    u'uid': u'1679',
                    u'user': u'andrewpmk',
                    u'version': u'7'},
    u'highway': u'tertiary',
    u'id': u'2627409',
    u'lanes': u'2',
    u'name': u'Mount Royal',
    u'node_refs': [   u'11753030',
                      u'11753034',
                      u'11753047',
                      u'1258712063',
                      u'11753061',
                      u'11753066'],
    u'type': u'way'}


This shows the distinct number of lanes present in 'way' tags.
As we can see it contains instances with more than one integers separated with semicolons.
We can see it as a shortcoming of the data as we could have converted these instances into lists with the probable lane numbers.

Different types of highways the 'way' tags contain:

In [35]:
print collection.distinct('highway', {'type':'way'})

[u'tertiary', u'residential', u'primary', u'motorway_link', u'motorway', u'unclassified', u'primary_link', u'secondary', u'construction', u'service', u'secondary_link', u'cycleway', u'tertiary_link', u'trunk', u'footway', u'trunk_link', u'track', u'path', u'razed', u'bridleway', u'raceway', u'pedestrian', u'living_street', u'steps', u'proposed', u'abandoned', u'rest_area', u'services', u'road', u'closed:cycleway']


Highway counts grouped by their types:

In [13]:
highway_count = collection.aggregate([{'$match':{'type':'way','highway':{'$exists':1}}},
                                      {'$group':{'_id':'$highway','count':{'$sum':1}}},{'$sort':{'count':-1}}])

In [14]:
for item in highway_count:
    print item

{u'count': 106509, u'_id': u'residential'}
{u'count': 41620, u'_id': u'service'}
{u'count': 23532, u'_id': u'footway'}
{u'count': 9785, u'_id': u'secondary'}
{u'count': 8472, u'_id': u'tertiary'}
{u'count': 3832, u'_id': u'primary'}
{u'count': 3760, u'_id': u'unclassified'}
{u'count': 3469, u'_id': u'path'}
{u'count': 2931, u'_id': u'motorway_link'}
{u'count': 1855, u'_id': u'motorway'}
{u'count': 1744, u'_id': u'secondary_link'}
{u'count': 1605, u'_id': u'primary_link'}
{u'count': 1195, u'_id': u'track'}
{u'count': 970, u'_id': u'cycleway'}
{u'count': 467, u'_id': u'trunk'}
{u'count': 456, u'_id': u'tertiary_link'}
{u'count': 416, u'_id': u'pedestrian'}
{u'count': 214, u'_id': u'trunk_link'}
{u'count': 214, u'_id': u'steps'}
{u'count': 180, u'_id': u'bridleway'}
{u'count': 46, u'_id': u'construction'}
{u'count': 34, u'_id': u'razed'}
{u'count': 31, u'_id': u'raceway'}
{u'count': 24, u'_id': u'proposed'}
{u'count': 22, u'_id': u'living_street'}
{u'count': 19, u'_id': u'rest_area'}
{u'c

Total node_refs present in way tags with highways:

In [21]:
total_node_refs_with_highways = collection.aggregate([{'$match':{'type':'way','highway':{'$exists':1}}},
                                                    {'$group':{'_id':'Highway_refs','total':{'$sum':{'$size':'$node_refs'}}}}
                                                    ])

In [22]:
for item in total_node_refs_with_highways:
    print item

{u'total': 1791254, u'_id': u'Highway_refs'}


Total node_refs present in each type of highway:

In [17]:
node_refs_groupedby_highway = collection.aggregate([{'$match':{'type':'way','highway':{'$exists':1}}},
                     {'$unwind':'$node_refs'},
                     {'$group':{'_id':'$highway','total_node_refs':{'$sum':1}}},{'$sort':{'total_node_refs':-1}}])

In [18]:
for item in node_refs_groupedby_highway:
    print item

{u'total_node_refs': 938716, u'_id': u'residential'}
{u'total_node_refs': 252845, u'_id': u'service'}
{u'total_node_refs': 189231, u'_id': u'footway'}
{u'total_node_refs': 82202, u'_id': u'tertiary'}
{u'total_node_refs': 69225, u'_id': u'path'}
{u'total_node_refs': 61058, u'_id': u'secondary'}
{u'total_node_refs': 37378, u'_id': u'unclassified'}
{u'total_node_refs': 28837, u'_id': u'motorway_link'}
{u'total_node_refs': 28560, u'_id': u'cycleway'}
{u'total_node_refs': 21271, u'_id': u'motorway'}
{u'total_node_refs': 19579, u'_id': u'primary'}
{u'total_node_refs': 16317, u'_id': u'track'}
{u'total_node_refs': 12621, u'_id': u'primary_link'}
{u'total_node_refs': 10474, u'_id': u'secondary_link'}
{u'total_node_refs': 8703, u'_id': u'bridleway'}
{u'total_node_refs': 4291, u'_id': u'pedestrian'}
{u'total_node_refs': 2896, u'_id': u'trunk'}
{u'total_node_refs': 2291, u'_id': u'tertiary_link'}
{u'total_node_refs': 1872, u'_id': u'trunk_link'}
{u'total_node_refs': 634, u'_id': u'raceway'}
{u'to

The Geographical location of Detroit,Michigan to be used for some of the following queries - LATITUDE = 42.331429, LONGITUDE = -83.045753

Documents which have no latitude or longitude:

In [5]:
print collection.find({'pos':[]}).count()

387910


Unsetting the field 'pos'(position) from such documents:

In [6]:
collection.update_many({'pos':[]},{'$unset':{'pos':''}})

It is verified that there are no more documents with no latitude and longitude given.

In [7]:
print collection.find({'pos':[]}).count()

0


Elements situated NORTH of the standard Detroit latitude:

In [61]:
print collection.find({'pos.0':{'$gt':'42.331429'}}).count()

2700545


Elements situated SOUTH of or ON the standard latitude of Detroit:

In [65]:
print collection.find({'pos.0':{'$lte':'42.331429'}}).count()

1137588


Elements due EAST of standard Detroit longitude: 

In [3]:
print collection.find({'pos.1':{'$gt':'-83.045753'}}).count()

2974847


Elements WEST of standard Detroit longitude:

In [4]:
print collection.find({'pos.1':{'$lte':'-83.045753'}}).count()

863286


The elements have been created by these distinct users:

In [19]:
print collection.distinct('created_by')

[u'JOSM', u'Merkaartor 0.12', u'r_coastlines', u'YahooApplet 1.0', u'Potlatch 0.8a', u'Potlatch 0.5d', u'Potlatch 0.8', u'Potlatch 0.8b', u'Potlatch 0.9', u'Potlatch 0.9a', u'Potlatch 0.10b', u'Potlatch 0.10', u'xybot', u'Potlatch 0.10e', u'Potlatch 0.10f', u'ailatix', u'Merkaartor 0.13', u'Potlatch alpha', u'Potlatch 0.9b', u'Potlatch 0.9c', u'Potlatch 0.7b', u'OSM Fixer', u'Potlatch 0.7', u'Potlatch 0.8c', u'Potlatch 0.10d', u'polyshp2osm-multipoly']


Number of documents which have a creator mentioned:

In [23]:
print collection.find({'created_by':{'$exists':1}}).count()

5829


Tags created by each user (Descending order of total tags created). A significantly large percentage of tags did not have any creator mentioned:

In [23]:
groups_by_creator = collection.aggregate([{'$group':{'_id':'$created_by', 'count':{'$sum':1}}},{'$sort':{'count':-1}}])
for item in groups_by_creator:
    print item

{u'count': 4220214, u'_id': None}
{u'count': 1799, u'_id': u'r_coastlines'}
{u'count': 1678, u'_id': u'JOSM'}
{u'count': 1314, u'_id': u'Merkaartor 0.12'}
{u'count': 399, u'_id': u'Merkaartor 0.13'}
{u'count': 172, u'_id': u'YahooApplet 1.0'}
{u'count': 97, u'_id': u'Potlatch 0.10f'}
{u'count': 97, u'_id': u'Potlatch alpha'}
{u'count': 95, u'_id': u'Potlatch 0.8a'}
{u'count': 51, u'_id': u'polyshp2osm-multipoly'}
{u'count': 28, u'_id': u'Potlatch 0.9c'}
{u'count': 26, u'_id': u'Potlatch 0.10e'}
{u'count': 23, u'_id': u'Potlatch 0.10b'}
{u'count': 11, u'_id': u'Potlatch 0.8b'}
{u'count': 7, u'_id': u'Potlatch 0.9'}
{u'count': 7, u'_id': u'Potlatch 0.8c'}
{u'count': 6, u'_id': u'Potlatch 0.7b'}
{u'count': 4, u'_id': u'Potlatch 0.9a'}
{u'count': 3, u'_id': u'Potlatch 0.9b'}
{u'count': 3, u'_id': u'Potlatch 0.10'}
{u'count': 2, u'_id': u'Potlatch 0.5d'}
{u'count': 2, u'_id': u'Potlatch 0.10d'}
{u'count': 1, u'_id': u'Potlatch 0.8'}
{u'count': 1, u'_id': u'xybot'}
{u'count': 1, u'_id': u'OS

Distinct streets present in our data, along with the count of how many unique of them there are:

In [24]:
distinct_streets = collection.distinct('address.street')
print distinct_streets
print 'Number of disctinct streets-',len(distinct_streets)

[u'W 26 Mile Road', u'Rockwood Street', u'E 12 Mile Road', u'Shimmons Road', u'Crooks Road', u'Rochester Road', u'E 15 Mile Road', u'E 9 Mile Road', u'Grove Street', u'Belleville Road', u'Sashabaw Road', u'E. Nevada Street', u'Ryan Road', u'Town Center Drive', u'Ouellette Avenue', u'S Grand Avenue', u'S Grand River Avenue', u'Orchard Lake Road', u'Hill Road', u'Lauzon Road', u'Southfield Road', u'Baldwin Avenue', u'W Warren Avenue', u'Fisher Road', u'Michigan Avenue', u'Fort Street', u'E Commerce Road', u'West Michigan Avenue', u'Dennison Street', u'Orchard Hill Place', u'W Maple Road', u'Livernois Road', u'Coolidge Road', u'E Maple Road', u'John R Road', u'W Long Lake Road', u'W Big Beaver Road', u'E Big Beaver Road', u'E Long Lake Road', u'Corporate Drive', u'Broadway Street', u'Sterns Road', u'Victoria Court', u'Hall Road', u'5 Mile Road', u'E. Tienken Road', u'Main Street', u'Old Milford Farms Drive', u'Dougall Avenue', u'Marquette Court', u'Van Dyke Avenue', u'Auburn Road', u'Dequ

Count of streets by their types: 

In [26]:
count_by_street_types = collection.aggregate([{'$match':{'address.street_type':{'$exists':1}}},
                                               {'$group':{'_id':'$address.street_type','count':{'$sum':1}}}])
for item in count_by_street_types:
    print item

{u'count': 32, u'_id': u'Drive'}
{u'count': 105, u'_id': u'Road'}
{u'count': 31, u'_id': u'Street'}
{u'count': 22, u'_id': u'Avenue'}
{u'count': 9, u'_id': u'Place'}
{u'count': 4, u'_id': u'Court'}


--------------------------------------------------------------------------------------------------------------------------------
The distinct address streets present in our data which are 110 in number.
Here we could have grouped the data by the type of streets i.e. Road, Street, Avenue, Court etc.
It would have been easier if we had built our JSON data keeping this in mind.
Otherwise it is difficult to group as it requires string parsing from a value inside a document.
We would have got a better idea of street types if we were able to do that.

In [22]:
list_lanes = collection.aggregate([{'$match':{'lanes':{'$size':2}}},{'$project':{'type':1, 'lanes':1}}])
for item in list_lanes:
    print item

{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fc50a'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fc9c8'), u'lanes': [u'1', u'2']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fcb1b'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fcc95'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fcc99'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fce27'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fcf05'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fcff0'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a597251362cbcb5fd079'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a598251362cbcb5fd4c9'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': ObjectId('5809a598251362cbcb5fd52c'), u'lanes': [u'2', u'1']}
{u'type': u'way', u'_id': Object

In [61]:
sample = collection.aggregate([{'$match':{'address.street':{'$exists':1}}},{'$limit':20}])

In [62]:
for item in sample:
    pp.pprint(item)

{   u'_id': ObjectId('5809a498251362cbcb272d09'),
    u'address': {   u'city': u'Washington',
                    u'housenumber': u'4505',
                    u'postcode': u'48094',
                    u'street': u'W 26 Mile Road',
                    u'street_type': u'Road'},
    u'created': {   u'changeset': u'3356653',
                    u'timestamp': u'2009-12-12T16:24:08Z',
                    u'uid': u'147510',
                    u'user': u'woodpeck_fixbot',
                    u'version': u'5'},
    u'id': u'61979974',
    u'pos': [u'42.7128371', u'-83.0680961'],
    u'type': u'node'}
{   u'_id': ObjectId('5809a4c1251362cbcb308d94'),
    u'address': {   u'housename': u'Jefferson Middle School',
                    u'housenumber': u'27900',
                    u'postcode': u'48081',
                    u'street': u'Rockwood Street',
                    u'street_type': u'Street'},
    u'amenity': u'school',
    u'created': {   u'changeset': u'14345137',
                    u'tim